# Function Calling with Mistral AI Models

* Mistral AI models can connect to external tools by a method called **function calling**. In this example, I will be connecting the **Mistral Small model** to two user defined functions.
* This example is targetted at **Shipping companies** whose customers want to know the status of their ordered items.

Mistral AI Models Available for function calling:
* Mistral Large
* Mistral Small
* Codestral
* Ministral 8B
* Ministral 3B
* Pixtral 12B
* Pixtral Large
* Mistral Nemo

Function Calling Steps:
* User: Specify tools and query
* Model: Generate function arguments if applicable
* User: Execute Function to obtain tool results
* Model: Generate Final Answer


Resources:
1. https://docs.mistral.ai/capabilities/function_calling/





In [1]:
# Importing the pandas library
import pandas as pd

# Assuming we have the following data
data = {
    'order_id': ['A0001', 'A0002', 'A0003', 'A0004', 'A0005'],
    'customer_id': ['CM001', 'CM002', 'CM003', 'CM002', 'CM001'],
    'shipping_cost': [50.50, 30.50, 20.60, 10.40, 35.20],
    'shipping_date': ['2025-10-05', '2025-10-06', '2025-10-07', '2025-10-05', '2025-10-08'],
    'shipping_status': ['Shipped', 'Processing', 'Delivered', 'Shipped', 'Processing']
}

# Create DataFrame
df = pd.DataFrame(data)
print(df)

  order_id customer_id  shipping_cost shipping_date shipping_status
0    A0001       CM001           50.5    2025-10-05         Shipped
1    A0002       CM002           30.5    2025-10-06      Processing
2    A0003       CM003           20.6    2025-10-07       Delivered
3    A0004       CM002           10.4    2025-10-05         Shipped
4    A0005       CM001           35.2    2025-10-08      Processing


## Step 1. User: specify tools and query
Tools:
* Shipping Status
* Shipping Date

User Query:
* What is the Status of my order? My order ID is A0002

### Tools
I considered the two functions below:
* retrieve_shipping_status
* retrieve_shipping_date



In [2]:
# Retrieve shipping status
def retrieve_shipping_status(df: data, order_id: str) -> str:
    if order_id in df.order_id.values: 
        return json.dumps({'status': df[df.order_id == order_id].shipping_status.item()})
    return json.dumps({'error': 'order id not found.'})

# Retrieve shipping date
def retrieve_shipping_date(df: data, shipping_id: str) -> str:
    if order_id in df.order_id.values: 
        return json.dumps({'date': df[df.order_id == order_id].shipping_date.item()})
    return json.dumps({'error': 'order id not found.'})

In [3]:
#  Describe the type, function name, function description, function parameters, and the required parameter for the two functions in JSON format
tools = [
    {
        "type": "function",
        "function": {
            "name": "retrieve_shipping_status",
            "description": "Get shipping status of order",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The order id.",
                    }
                },
                "required": ["order_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_shipping_date",
            "description": "Get shipping date of a order",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The order id.",
                    }
                },
                "required": ["order_id"],
            },
        },
    }
]

In [4]:
# Create Dictionary of the two Functions
import functools

names_to_functions = {
    'retrieve_shipping_status': functools.partial(retrieve_shipping_status, df=df),
    'retrieve_shipping_date': functools.partial(retrieve_shipping_date, df=df)
}

### User Query

In [5]:
messages = [{"role": "user", "content": "What's the status of my order A0002?"}]

## Step 2. Model: Generate function arguments

* Provide both the user query and tools specification to the mistral model. 

The goal of this step is to:
1. Determine the appropriate function to use.
2. identify if there is any essential information missing for a function.
3. generate necessary arguments for the chosen function.

In [ ]:
from mistralai import Mistral

api_key ='API_KEY'
model = "mistral-small-latest"

client = Mistral(api_key=api_key)
response = client.chat.complete(
    model = model,
    messages = messages,
    tools = tools,
    tool_choice = "any", # Forces tool use 
)
response

ChatCompletionResponse(id='8b9d2954f70a4edf8bcf727ae16a5a9e', object='chat.completion', model='mistral-small-latest', usage=UsageInfo(prompt_tokens=156, completion_tokens=27, total_tokens=183), created=1740148855, choices=[ChatCompletionChoice(index=0, message=AssistantMessage(content='', tool_calls=[ToolCall(function=FunctionCall(name='retrieve_shipping_status', arguments='{"order_id": "A0002"}'), id='cRF8ozJlV', type=None, index=0)], prefix=False, role='assistant'), finish_reason='tool_calls')])

In [7]:
# Add response to the message list
messages.append(response.choices[0].message)

## Step 3. User: Execute function to obtain tool results

In [8]:
#Extract the function name and parameters from the response
import json

tool_call = response.choices[0].message.tool_calls[0]
function_name = tool_call.function.name
function_params = json.loads(tool_call.function.arguments)
print("\nfunction_name: ", function_name, "\nfunction_params: ", function_params)


function_name:  retrieve_shipping_status 
function_params:  {'order_id': 'A0002'}


In [9]:
#Execute the function to get the output
function_result = names_to_functions[function_name](**function_params)
function_result

'{"status": "Processing"}'

## Step 4. Model: Generate final answer

In [10]:
# Final response of the model
messages.append({"role":"tool", "name":function_name, "content":function_result, "tool_call_id":tool_call.id})

response = client.chat.complete(
    model = model, 
    messages = messages
)
response.choices[0].message.content

'Your order A0002 is currently being processed.'

 TIP: You can try other models and adapt this example for other use cases just like I customized mine targeting shipping companies.